In [23]:

import pandas as pd
import numpy as np

from src.utils import remove_brackets_in_categorical_values
from src.data_management.cleaning_helpers.renaming_replacing import set_types_file
from src.data_management.cleaning_helpers.data_checks import general_data_checks

In [2]:
df=pd.read_stata("src/original_data/HHENDDAT_cf_W11.dta",convert_categoricals=False)

pd.read_csv("src/data_management/PENDDAT/penddat_renaming.csv",sep=";")

In [32]:
new_names=pd.read_csv("src/data_management/PENDDAT/penddat_renaming.csv",sep=";")["new_name"]

In [34]:
renaming_dict=dict(zip(df.columns,new_names))

In [43]:
pd.isna(renaming_dict)

False

In [41]:
df.rename(columns=renaming_dict)

{'hnr': 'p_id',
 'welle': 'hh_id',
 'hintjahr': 'wave',
 'hintmon': 'survey_year',
 'HW0300': 'survey_mon',
 'HA0100': 'sex',
 'HLS0100a': 'age',
 'HLS0100b': 'religiosity',
 'HLS0200a': nan,
 'HLS0200b': nan,
 'HLS0300a': nan,
 'HLS0300b': nan,
 'HLS0400a': nan,
 'HLS0400b': nan,
 'HLS0500a': nan,
 'HLS0500b': nan,
 'HLS0600a': nan,
 'HLS0600b': nan,
 'HLS0700a': nan,
 'HLS0700b': nan,
 'HLS0800a': nan,
 'HLS0800b': nan,
 'HLS0900a': nan,
 'HLS0900b': nan,
 'HLS1000a': nan,
 'HLS1000b': nan,
 'HLS1100a': nan,
 'HLS1100b': nan,
 'HLS1200a': nan,
 'HLS1200b': nan,
 'HLS1300a': nan,
 'HLS1300b': nan,
 'HLS1400a': nan,
 'HLS1400b': nan,
 'HLS1500a': nan,
 'HLS1500b': nan,
 'HLS1600a': nan,
 'HLS1600b': 'b5_ext_n_a',
 'HLS1700a': 'b5_agree_n_a',
 'HLS1700b': 'b5_consc_a',
 'HLS1800a': 'b5_neu_a',
 'HLS1800b': 'b5_open_a',
 'HLS1900a': 'b5_ext_a',
 'HLS1900b': 'b5_agree_a',
 'HLS2000a': 'b5_consc_n_a',
 'HLS2000b': 'b5_neu_n_a',
 'HLS2100a': 'b5_open_b',
 'HLS2100b': 'b5_ext_n_b',
 'HLS2200

In [9]:
def _fix_nans(x):
    if pd.isna(x):
        return np.nan
    else:
        return x

In [13]:
def _check_for_two_types_of_missing(df):
    # Make sure columns contain only one type of missing
    for col in df.columns:
        pd_na = any(x is pd.NA for x in list(df[col].unique()))
        np_na = any(x is np.nan for x in list(df[col].unique()))
        if pd_na & np_na:
            print(
                f"{col} contains two types of nans. Will be fixed. Should be checked!"
            )
            df[col] = df[col].map(lambda x: _fix_nans(x))
    return df

In [20]:
def replace_values(panel, replace_dict, rename_df):
    """Replace and rename values using the replace dictionary.

    Args:
        panel (pandas.DataFrame): The dataframe which values need to be
            replaced or renamed.
        replace_dict (dictionary): The replacing dictionary.
        rename_df (pandas.DataFrame): The renaming dataframe taken from the
            renaming file.

    Returns:
        pandas.DataFrame: The dataframe with the replaced or renamed values.

    """

    out = panel.copy()
    # Convert some columns to lower case
    if "mixed_case" in replace_dict and replace_dict["mixed_case"]:
        out[replace_dict["mixed_case"]] = out[replace_dict["mixed_case"]].apply(
            lambda x: x.str.lower()
        )

    # Convert numeric columns
    if "numeric" in replace_dict and replace_dict["numeric"]:
        out[replace_dict["numeric"]] = out[replace_dict["numeric"]].apply(
            lambda x: pd.to_numeric(x, errors="coerce")
        )

    # Rename variables according to their types.
    if "type renaming" in replace_dict:

        rename_df["type"] = rename_df["type"].replace(
            {
                "int": "Int64",
                "float": "float64",
                "bool": "boolean",
                "Categorical": "category",
                "Int": "Int64",
            }
        )

        for group in replace_dict["type renaming"] and replace_dict["type renaming"]:
            filter = rename_df["type"] == group
            cols = rename_df.copy()[filter]["new_name"].values
            for col in cols:
                try:
                    out[col] = out[col].replace(replace_dict["type renaming"][group])
                except Exception:
                    print(f"issue with {col}")
                    continue

    # Rename variables in multiple columns.
    if "multicolumn" in replace_dict and replace_dict["multicolumn"]:
        for _j in replace_dict["multicolumn"]:
            try:
                out.loc[:, replace_dict["multicolumn"][_j]["columns"]] = out.loc[
                    :, replace_dict["multicolumn"][_j]["columns"]
                ].replace(replace_dict["multicolumn"][_j]["dictionary"])
            except Exception:
                print(f"error in {replace_dict['multicolumn'][_j]}")

    # Rename variables according to the renaming dictionary
    if "replacing" in replace_dict and replace_dict["replacing"]:
        for _i in replace_dict["replacing"]:
            if _i != "full_df":
                try:
                    out[_i].replace(replace_dict["replacing"][_i], inplace=True)
                except TypeError:
                    print(f"type issue with {_i}")
            else:
                try:
                    out.replace(replace_dict["replacing"][_i], inplace=True)
                except TypeError:
                    print(f"type issue with {_i}")

    out = _check_for_two_types_of_missing(out)

    return out

In [ ]:
def logical_cleaning(panel, logical_cleaning_dict):
    """Some logical cleaning specified in logical_cleaning_dict.

    Args:
        panel (pandas.DataFrame): The dataframe which values need to be
            replaced or renamed.
        logical_cleaning_dict (dictionary): The specificaiton dictionary.


    Returns:
        pandas.DataFrame: The dataframe with the replaced or renamed values.

    """
    out = panel.copy()

    # Fill nans
    if "fillna" in logical_cleaning_dict and logical_cleaning_dict["fillna"]:
        for col, value in logical_cleaning_dict["fillna"].items():
            print(col, out[col].dtype)
            out[col] = out[col].fillna(value)

    return out

In [ ]:
def _clean_logically_pl(data, logical_cleaning_dict):
    """Clean some of the data logically
    Args:
        data(pandas.DataFrame): The data frame to be cleaned.
        logical_cleaning_dict (dictionary): The specification dictionary for automated
                                            logical cleaning.
    Returns:
        pandas.DataFrame: The logically cleaned data frame.
    """
    out = data.copy()
    out = logical_cleaning(out, logical_cleaning_dict)
    # Divide by 100 because it contains an error;
    # now same scale as prv_rente_beitr_2013_m
    out["prv_rente_beitr_2018_m"] = out["prv_rente_beitr_2018_m"] / 100
    # Calculate average mothly payments into private pension for years 2013 & 2018
    out["prv_rente_beitr_2013_m"] = (
        out["prv_rente_beitr_2013_m"] * out["in_priv_rente_eingezahlt_monate"] / 12
    )
    out["prv_rente_beitr_2018_m"] = (
        out["prv_rente_beitr_2018_m"] * out["in_priv_rente_eingezahlt_monate"] / 12
    )
    # Set average mothly payments = 0 when respondent said she
    # didnt pay into private pension
    out.loc[
        out["in_priv_rente_eingezahlt"] == "Nein",
        ["prv_rente_beitr_2013_m", "prv_rente_beitr_2018_m"],
    ] = 0
    #
    out["prv_rente_beitr_m"] = out["prv_rente_beitr_2013_m"]
    out.loc[out["jahr"] == 2018, "prv_rente_beitr_m"] = out.loc[
        out["jahr"] == 2018, "prv_rente_beitr_2018_m"
    ]
    # Health variables and Frailty index
    med_vars = [
        "med_pl_schw_treppen",
        "med_pl_schw_taten",
        "med_pl_schlaf",
        "med_pl_diabetes",
        "med_pl_asthma",
        "med_pl_herzkr",
        "med_pl_krebs",
        "med_pl_schlaganf",
        "med_pl_migraene",
        "med_pl_bluthdrck",
        "med_pl_depressiv",
        "med_pl_demenz",
        "med_pl_gelenk",
        "med_pl_ruecken",
        "med_pl_sonst",
        "med_pl_raucher",
        "med_pl_subj_status",
    ]
    out[med_vars] = out[med_vars].astype(float)
    out[med_vars] = out.groupby("p_id")[
        med_vars
    ].ffill()  # fill gaps in between surveys with previous values
    out["bmi_pl"] = out["med_pl_gewicht"] / ((out["med_pl_groesse"] / 100) ** 2)
    out["bmi_pl_dummy"] = (out["bmi_pl"] >= 30).astype(float)
    out["med_pl_subj_status_dummy"] = (out["med_pl_subj_status"] >= 3).astype(float)

    med_vars.append("bmi_pl_dummy")
    med_vars.append("med_pl_subj_status_dummy")
    med_vars.remove("med_pl_subj_status")

    out["frailty_pl"] = out[med_vars].mean(axis=1)
    return out

In [ ]:
def _replace_values_pl(data, replace_dict, rename_df):

    data = remove_brackets_in_categorical_values(data)

    # General replacing
    data = replace_values(data, replace_dict, rename_df)

    return data

In [ ]:
def _check_pl(data):
    """Check some of the data in the work_schooling database.
    Args:
        data(pandas.DataFrame): The data frame to be checked.
    """
    out = data.copy()
    general_data_checks(out)

In [21]:
def clean_hhenddat(data, rename_df, cleaning_specs):

    # Replace values
    data = _replace_values_pl(data, cleaning_specs["replacing"], rename_df)

    # Set types of variables using renaming file.
    data = set_types_file(
        panel=data,
        rename_df=rename_df,
        cat_sep="|",
        int_to_float=True,
        bool_to_float=True,
    )

    # Logical cleaning of work schooling
    data = _clean_logically_pl(data, cleaning_specs["logical_cleaning"])

    # Check some consistency in the data.
    _check_pl(data)

    return data

In [ ]:
def replace_values(panel, replace_dict, rename_df):
    """Replace and rename values using the replace dictionary.

    Args:
        panel (pandas.DataFrame): The dataframe which values need to be
            replaced or renamed.
        replace_dict (dictionary): The replacing dictionary.
        rename_df (pandas.DataFrame): The renaming dataframe taken from the
            renaming file.

    Returns:
        pandas.DataFrame: The dataframe with the replaced or renamed values.

    """

    out = panel.copy()
    # Convert some columns to lower case
    if "mixed_case" in replace_dict and replace_dict["mixed_case"]:
        out[replace_dict["mixed_case"]] = out[replace_dict["mixed_case"]].apply(
            lambda x: x.str.lower()
        )

    # Convert numeric columns
    if "numeric" in replace_dict and replace_dict["numeric"]:
        out[replace_dict["numeric"]] = out[replace_dict["numeric"]].apply(
            lambda x: pd.to_numeric(x, errors="coerce")
        )

    # Rename variables according to their types.
    if "type renaming" in replace_dict:

        rename_df["type"] = rename_df["type"].replace(
            {
                "int": "Int64",
                "float": "float64",
                "bool": "boolean",
                "Categorical": "category",
                "Int": "Int64",
            }
        )

        for group in replace_dict["type renaming"] and replace_dict["type renaming"]:
            filter = rename_df["type"] == group
            cols = rename_df.copy()[filter]["new_name"].values
            for col in cols:
                try:
                    out[col] = out[col].replace(replace_dict["type renaming"][group])
                except Exception:
                    print(f"issue with {col}")
                    continue

    # Rename variables in multiple columns.
    if "multicolumn" in replace_dict and replace_dict["multicolumn"]:
        for _j in replace_dict["multicolumn"]:
            try:
                out.loc[:, replace_dict["multicolumn"][_j]["columns"]] = out.loc[
                    :, replace_dict["multicolumn"][_j]["columns"]
                ].replace(replace_dict["multicolumn"][_j]["dictionary"])
            except Exception:
                print(f"error in {replace_dict['multicolumn'][_j]}")

    # Rename variables according to the renaming dictionary
    if "replacing" in replace_dict and replace_dict["replacing"]:
        for _i in replace_dict["replacing"]:
            if _i != "full_df":
                try:
                    out[_i].replace(replace_dict["replacing"][_i], inplace=True)
                except TypeError:
                    print(f"type issue with {_i}")
            else:
                try:
                    out.replace(replace_dict["replacing"][_i], inplace=True)
                except TypeError:
                    print(f"type issue with {_i}")

    out = _check_for_two_types_of_missing(out)

    return out

In [35]:
pd.read_csv("src/data_management/HHENDDAT/hhenddat_renaming.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'src/data_management/HHENDDAT/hhenddat_renaming.csv'